## Load data

In [1]:
import config

from navernews_10days import get_comments_paths
from soynlp.noun import LRNounExtractor_v2
from soynlp.tokenizer import LTokenizer
from soynlp.utils import DoublespaceLineCorpus

path = get_comments_paths(date='2016-10-20')
corpus = DoublespaceLineCorpus(path, iter_sent=True)

soynlp=0.0.493
added lovit_textmining_dataset


## Train Word Extractor and Tokenizer

명사 추출기를 이용하여 명사 점수를 단어 점수로 이용합니다. 뉴스 기사는 띄어쓰기가 잘 되어 있기 때문에 L-Tokenizer 를 이용하였습니다.

In [2]:
noun_extractor = LRNounExtractor_v2()
noun_scores = noun_extractor.train_extract(corpus)

word_scores = {noun:score.score for noun, score in noun_scores.items()}
tokenizer = LTokenizer(scores = word_scores)

tokenizer.tokenize('뉴스 기사를 이용하여 학습한 모델입니다', tolerance=0.3)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 687560 from 228630 sents. mem=0.198 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=2775027, mem=1.670 Gb
[Noun Extractor] batch prediction was completed for 193198 words
[Noun Extractor] checked compounds. discovered 151319 compounds
[Noun Extractor] postprocessing detaching_features : 167318 -> 120870
[Noun Extractor] postprocessing ignore_features : 120870 -> 120447
[Noun Extractor] postprocessing ignore_NJ : 120447 -> 117740
[Noun Extractor] 117740 nouns (151319 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=2.027 Gb                    
[Noun Extractor] 69.13 % eojeols are covered


['뉴스', '기사', '를', '이용', '하여', '학습', '한', '모델', '입니다']

## Build co-occurrence matrix

L parts 에서 명사 점수의 최대값과 0.3 점 차이가 나지 않는다면 그 중 가장 긴 subword 를 단어로 추출하도록 tolerance 를 이용하였습니다. min_tf 를 10 으로 설정하여 10 번 이하로 등장한 단어에 대해서는 co-occurrence 를 계산하지 않습니다.

In [3]:
from soynlp.vectorizer import sent_to_word_contexts_matrix

def custom_tokenizer(sent):
    return tokenizer.tokenize(sent, tolerance=0.3)

x, idx2vocab = sent_to_word_contexts_matrix(
    corpus,
    windows = 2,
    min_tf = 10,
    tokenizer = custom_tokenizer,
    verbose = True
)

Create (word, contexts) matrix
  - counting word frequency from 228629 sents, mem=2.172 Gb
  - scanning (word, context) pairs from 228629 sents, mem=2.172 Gb
  - (word, context) matrix was constructed. shape = (30618, 30618)                    
  - done


학습 후 30,810 개의 단어가 학습되었습니다.

In [4]:
x.shape

(30618, 30618)

## similar words using context vector

idx2vocab 을 vocab2idx 로 만듭니다. 

In [5]:
vocab2idx = {vocab:idx for idx, vocab in enumerate(idx2vocab)}

print(vocab2idx['이화여대'])
print(vocab2idx['아이오아이'])

1340
1984


sklearn.metrics.pairwise_distances 를 이용하여 '이화여대'와 context vector 가 비슷한 다른 단어를 찾습니다. 

numpy.ndarray 형식인 distance matrix (dist) 에 대하여 argsort() 를 하면, 거리 순서대로 정렬됩니다. sort() 를 하면 값이 정렬되며, argsort() 를 하면 각 값의 index 가 return 됩니다. 

여러 번 쓸 수 있도록 함수로도 만들어둡니다. 

In [6]:
import numpy as np
from sklearn.metrics import pairwise_distances

def most_similar_words(query, wv, topk=10):
    """
    query : str
    wv : word representation
    topk : num of similar terms
    """
    
    if not (query in vocab2idx):
        return []

    query_idx = vocab2idx[query]
    query_vec = wv[query_idx,:].reshape(1,-1)
    dist = pairwise_distances(query_vec, wv, metric='cosine')[0]
    similars = []

    # sorting
    for similar_idx in dist.argsort():
        
        # filtering query term
        if similar_idx == query_idx:
            continue

        if len(similars) >= topk:
            break

        # decoding index to word
        similar_word = idx2vocab[similar_idx]
        similars.append((similar_word, 1 - dist[similar_idx]))

    return similars

Cooccurrence matrix 만을 이용해도 문맥이 **매우** 확실한 단어 `아이오아이`는 다른 아이돌 이름이나 `아오아` 같은 약어가 유사어로 검색됩니다.

In [7]:
most_similar_words('아이오아이', x)

[('블랙핑크', 0.6581659649921858),
 ('트와이스', 0.6258572495971212),
 ('엑소', 0.5828096369899102),
 ('여자친구', 0.5788681011489352),
 ('노래', 0.5463866900815891),
 ('우리나라', 0.5435773641801577),
 ('아오아', 0.5379811668561362),
 ('우리', 0.5273834320883796),
 ('가수', 0.5266958156561162),
 ('새누리', 0.5220714249424844)]

그러나 단어 `아프리카`의 유사어는 잘 검색되지 않습니다. Co-occurrence frequency 를 representation 으로 이용하면 어느 문맥에나 등장하는 단어들에 영향을 받기 때문입니다.

In [8]:
most_similar_words('아프리카', x)

[('회사', 0.8037971762424634),
 ('우리나라', 0.7900979733733253),
 ('머리', 0.78224110041578),
 ('우리', 0.7763852370588487),
 ('이나라', 0.764068648599605),
 ('정부', 0.7632069770872995),
 ('학교', 0.7615474449969353),
 ('너네', 0.7518707053229157),
 ('스크린도어', 0.7501438876269355),
 ('새누리', 0.7406757432943758)]

## PPMI 를 이용한 term weighting

soynlp 의 pmi 를 이용하여 co-occurrence matrix 에 PMI 를 적용합니다. `min_pmi` 를 0 으로 설정하여 Positive PMI 로 만듭니다.

In [9]:
from soynlp.word import pmi

x_pmi, px, py = pmi(x, min_pmi=0.0)

PPMI 가 어느 문맥에나 등장하는 단어의 weight 를 0 으로 만들기 때문에 단어의 문맥이 뚜렷해집니다. `아이오아이`의 유사어는 크게 달라지지 않았습니다.

In [10]:
most_similar_words('아이오아이', x_pmi)

[('ioi', 0.17973380445809284),
 ('블랙핑크', 0.16560387184185066),
 ('트와이스', 0.15580021478108652),
 ('아오아', 0.14810905975425448),
 ('블핑', 0.1361151825149176),
 ('너무너무너무', 0.13231819811777146),
 ('CLC', 0.12497139230114185),
 ('엑소', 0.12160576569048331),
 ('방탄', 0.11965180115621088),
 ('트와', 0.11833934686696168)]

그러나 `아프리카`의 유사어는 크게 바뀌었습니다. 다른 나라의 이름도 검색되며, `아프리카tv` 관련 단어인 `bj` 나 `유튜브`가 검색됩니다. 이날 대륙 `아프리카`에 관련된 기사와 `아프리카tv` 관련 기사가 모두 있었기 때문에 두 문맥이 모두 반영되어 있습니다.

그러나 representation vector 의 차원은 3 만 차원보다 큽니다.

In [11]:
most_similar_words('아프리카', x_pmi)

[('비제이', 0.08428223905858767),
 ('bj', 0.07921126656182187),
 ('수수료', 0.0742366382647287),
 ('중동', 0.07224957662291298),
 ('유튜브', 0.06996363416062701),
 ('플랫폼', 0.06782625167091183),
 ('대도서관', 0.06712023146678547),
 ('벌어들', 0.06647189913183482),
 ('유투브', 0.06417289761041922),
 ('클린', 0.061716503788260324)]

## PPMI + SVD 를 이용한 차원 축소

Singular Vector Decomposition (SVD) 를 이용하여 차원을 축소합니다.

In [12]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=300)
x_pmisvd = svd.fit_transform(x_pmi)

SVD 로 차원 축소를 하면 noise 가 어느 정도 제거됩니다. Co-occurrence matrix 에서의 noise 는 문맥이 특이하거나 infrequent 한 단어입니다. 그 외에는 PPMI 를 적용했을 때와 유사어의 관계는 비슷합니다. 하지만 cosine 유사도의 값은 커집니다. PPMI 만 적용하면 sparse vector 이기 때문에 평균적인 유사도가 작습니다. 하지만 SVD 를 적용하면 평균적인 유사도 scale 이 커집니다.

In [13]:
most_similar_words('아이오아이', x_pmisvd)

[('블랙핑크', 0.7965188797621157),
 ('트와이스', 0.7951284625715093),
 ('블핑', 0.7740097675003359),
 ('트와', 0.7493663315577943),
 ('엑소', 0.7408083642013694),
 ('ioi', 0.7250002122971044),
 ('빅뱅', 0.7234435047012071),
 ('아오아', 0.7097597687481514),
 ('여자친구', 0.704434021408181),
 ('방탄', 0.6874623193165361)]

In [14]:
most_similar_words('아프리카', x_pmisvd)

[('유투브', 0.4899624680949647),
 ('유튜브', 0.4607744719229442),
 ('bj', 0.45474446236500043),
 ('프로그램', 0.43869881299333),
 ('한계', 0.43799992904494467),
 ('중소기업', 0.4284874079048935),
 ('후진국', 0.4225473546074232),
 ('동남아', 0.42075192796849603),
 ('이슬람', 0.4187904041774626),
 ('이미지', 0.4114119571148269)]

## S, Sigma, V 를 이용하여 representation 과 mapper 만들기

In [15]:
from sklearn.utils import check_random_state
from sklearn.utils.extmath import randomized_svd

def train(X, n_components, n_iter=5, random_state=None):

    if (random_state == None) or isinstance(random_state, int):
        random_state = check_random_state(random_state)

    n_features = X.shape[1]

    if n_components >= n_features:
        raise ValueError("n_components must be < n_features;"
                         " got %d >= %d" % (n_components, n_features))

    U, Sigma, VT = randomized_svd(
        X, n_components,
        n_iter = n_iter,
        random_state = random_state)

    S_ = Sigma ** (0.5)
    representation = y = U * S_
    mapper = (VT.T * S_).T

    return representation, mapper

wv, mapper = train(x_pmi, n_components=300)

In [16]:
most_similar_words('아이오아이', wv)

[('트와이스', 0.7562570038775457),
 ('블랙핑크', 0.7542341700254398),
 ('엑소', 0.679682713038533),
 ('트와', 0.6700636725186646),
 ('블핑', 0.6696587966066099),
 ('ioi', 0.6506407392295206),
 ('빅뱅', 0.6324141740977194),
 ('여자친구', 0.6268380730416317),
 ('아오아', 0.6158338023198218),
 ('유닛', 0.6025530918570341)]

In [17]:
most_similar_words('아프리카', wv)

[('유투브', 0.3864460957574195),
 ('티비', 0.35121599517166446),
 ('bj', 0.33680517731984305),
 ('유튜브', 0.3335360050218502),
 ('비제이', 0.3188627913806349),
 ('중동', 0.30562114059481327),
 ('동남아', 0.297176342770448),
 ('시청자', 0.28850889710176764),
 ('tv', 0.2873335460623294),
 ('이슬람', 0.2803188144537745)]